In [1]:
import pandas as pd
import numpy as np
from openpyxl import load_workbook
from IPython.core.display import display, HTML
from pathlib import Path
import json

In [2]:
css_rules = Path('dataframe.css').read_text()
HTML('<style>' + css_rules + '</style>')

In [3]:
source = './data-input.xlsx'
source_filter = 'Eth'

In [4]:
def generate_json_file(data, name):
    name = name.replace(" ","_").lower()
    json_object = json.dumps(data, indent = 4)
    with open(f"../backend/source/form_{name}.json", "w") as outfile:
        outfile.write(json_object)

In [5]:
def get_definitions(data, form_name, location):
    forms = []
    jsonforms = []
    for index, col in enumerate(list(data)):
        datatype = data[col].dtypes
        formtype = "text"
        options = None
        if datatype == int:
            formtype = "number"
        if datatype == np.float64:
            formtype = "number"
        if datatype == object:
            test = data[col].dropna()
            test = test.str.lower()
            options = list(test.unique())
            if len(options) > 8:
                options = None
                formtype = "text"
            else:
                formtype = "option"
                if len(options) == 1:
                    for yn in ["yes","no"]:
                        if options[0].lower() == yn:
                            options = ["yes","no"]
                else:
                    options = [str(o).lower() for o in options]
        if col in location:
            options = None
            formtype = "administration"
        cname = col.replace("_"," ").lower().strip()
        if "|" in cname:
            cname = cname.split("|")[1].strip()
        if formtype == "option":
            jsonforms.append({"order": index + 1,"question": cname, "type": formtype, "options": options})
            for opt in options:
                forms.append({"ID": index + 1,"QUESTION": cname, "TYPE": formtype.upper(), "OPTIONS": opt})
        else:
            jsonforms.append({"order": index + 1,"question": cname, "type": formtype, "options": None})
            forms.append({"ID":  index + 1, "QUESTION": cname, "TYPE": formtype.upper(), "OPTIONS": " - "})
    generate_json_file(jsonforms, form_name)
    results = pd.DataFrame(forms).groupby(['ID','QUESTION','TYPE','OPTIONS']).first()
    display(HTML(f"<h1>{form_name}</h1>"))
    display(HTML(results.to_html()))
    display(HTML(f"<hr/>"))
    return results

In [6]:
all_sheets = load_workbook(source, read_only=True).sheetnames
sheets = list(filter(lambda x: 'Eth' in x, all_sheets))

for sheet in sheets:
    data = pd.read_excel(source, sheet)
    data.drop(data.filter(regex="Unnamed"),axis=1, inplace=True)
    get_definitions(data, sheet, ["Woreda","Kebele"])

,,,
ID,QUESTION,TYPE,OPTIONS
1,woreda,ADMINISTRATION,-
2,kebele,ADMINISTRATION,-
3,village,TEXT,-
4,latitude,NUMBER,-
5,longitude,NUMBER,-
6,no. of hhs,NUMBER,-
7,initial number latrines,NUMBER,-
8,final number of latrines,NUMBER,-
9,date triggered,TEXT,-
